# Comparing previous SID convergence with TSSOS #

In [1]:
# using packages
using DynamicPolynomials
using LinearAlgebra
using HDF5
using TSSOS
using Dates
using ProgressBars

Declare unknown symbolic matrices to build dynamic equations

In [2]:
# unknown Hermitian 2 x 2 matrix
@polyvar x[1:4]
H = [ 1.0 * x[1]        x[3] + im * x[4]
      x[3] - im * x[4]  x[2]             ]

# unknown arbitrary complex 2 x 2 matrix
@polyvar a[1:2, 1:2]
@polyvar b[1:2, 1:2]
A = 1.0 * a + im * b

2×2 Matrix{Polynomial{true, ComplexF64}}:
 a₁₋₁ + (0.0+1.0im)b₁₋₁  a₁₋₂ + (0.0+1.0im)b₁₋₂
 a₂₋₁ + (0.0+1.0im)b₂₋₁  a₂₋₂ + (0.0+1.0im)b₂₋₂

Function to build left hand side of the Lindblad equation

In [3]:
function lindblad_rhs(rho)
    return -1.0 * im * (H * rho - rho * H) + A * rho * A' - (A' * A  * rho + rho * A' * A) / 2
end

lindblad_rhs (generic function with 1 method)

Function to calculate square of Frobenius norm of the matrix

In [4]:
function frobenius_norm2(m)
    return tr(m * m')
end

frobenius_norm2 (generic function with 1 method)

Function to get seeds from the HDF5 data file for specific noise level <em> w </em>

In [5]:
function get_seeds(file_name, w)
    h5open(file_name, "r") do file
        return keys(file[string(w)])
    end
end    

get_seeds (generic function with 1 method)

In [6]:
function get_noise_levels(file_name)
    h5open(file_name, "r") do file
        return keys(file)
    end
end

get_noise_levels (generic function with 1 method)

Function to build Pade objective

In [7]:
function pade_obj(ρ_series, time_series)
    dt = diff(time_series)
    obj = sum(
        frobenius_norm2(
            ρ_series[:, :, t] - ρ_series[:, :, t - 1] 
            - dt[t-1] * lindblad_rhs(0.5 * (ρ_series[:, :, t] + ρ_series[:, :, t - 1]))
        ) for t in 2:size(ρ_series,3)
    )
    return obj
end    

pade_obj (generic function with 1 method)

Function to find the best(minimum) value of the objective polynomial

In [8]:
function get_rho_series(file_name, w, seed)
    h5open(file_name, "r") do file
        dset = file[string(w)][string(seed)]["rho"]
        return(read(dset))
    end
end

function opt_tssos(ρ_series)
    
    dt = 0.5
    
    # objective for optimization 
      
    obj = sum(
        frobenius_norm2(
            ρ_series[:, :, t] - ρ_series[:, :, t - 1] 
            - dt * lindblad_rhs(0.5 * (ρ_series[:, :, t] + ρ_series[:, :, t - 1]))
        ) for t in 2:size(ρ_series,3)
    )

    
    obj_re = sum(real(coef) * mon for (coef, mon) in zip(coefficients(obj), monomials(obj)))
    
    # execute the first level of the TSSOS hierarchy

    opt,sol,data = tssos_first(obj_re, variables(obj_re))
    
    # execute higher levels of the TSSOS hierarchy
    
    iter = 1
    first_opt = opt
    best_opt = opt

    while !isnothing(opt)
        opt,sol,data = tssos_higher!(data)
        if !isnothing(opt) & (iter < 5)
            iter += 1
            if abs(opt) < abs(best_opt)
                best_opt = opt
            end
        end
    end
    
    return(first_opt, best_opt, iter)
    
end    

opt_tssos (generic function with 1 method)

Reading time series of the dynamics of density matrix $\rho$ from HDF file and calculating optimums with TSSOS

In [9]:
file_name = "time_series_lindblad_rnd_rho_2022-04-27_03-35-20.hdf5"

noise_levels = get_noise_levels(file_name)

output_file_name = "opts" * Dates.format(now(), "yyyy-mm-dd_HH-MM-SS") * ".hdf5"

for w in noise_levels
    
    println("Processing w = ", w)
    
    seeds = get_seeds(file_name, w)
    
    h5open(output_file_name, "cw") do out_file
        create_group(out_file, string(w))
    end
    
    
    for seed in ProgressBar(seeds)
        
        println("SEED="*seed)
        
        ρ_series = get_rho_series(file_name, w, seed)      
        first_opt, best_opt, iter = opt_tssos(ρ_series)
 
        h5open(output_file_name, "cw") do out_file  # read-write, create file if not existing, preserve existing contents
            w_group = open_group(out_file, string(w))
            seed_group = create_group(w_group, string(seed))
            write_dataset(seed_group, "first_opt", first_opt)
            write_dataset(seed_group, "best_opt", best_opt)
            write_dataset(seed_group, "iter", iter)
        end      
    end
end

Processing w = 0.0
SEED=1000923797
************************TSSOS************************
TSSOS is launching...
Starting to compute the block structure...
------------------------------------------------------
The sizes of PSD blocks:
[41, 8]
[1, 1]
------------------------------------------------------
Obtained the block structure. The maximal size of blocks is 41.
Assembling the SDP...
There are 525 affine constraints.
Solving the SDP...
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 525             
  Cones                  : 0               
  Scalar variables       : 1               
  Matrix variables       : 2               
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator t

Factor     - setup time             : 0.02              dense det. time        : 0.00            
Factor     - ML order time          : 0.00              GP order time          : 0.00            
Factor     - nonzeros before factor : 1.38e+05          after factor           : 1.38e+05        
Factor     - dense dim.             : 0                 flops                  : 5.04e+07        
ITE PFEAS    DFEAS    GFEAS    PRSTATUS   POBJ              DOBJ              MU       TIME  
0   3.0e+01  1.0e+00  1.0e+00  0.00e+00   0.000000000e+00   0.000000000e+00   1.0e+00  0.02  
1   1.3e+01  4.3e-01  6.3e-01  -9.64e-01  -1.437891326e+00  -2.594712177e+00  4.3e-01  0.05  
2   2.7e+00  8.9e-02  2.3e-01  -8.72e-01  -2.096114474e+01  -2.693946553e+01  8.9e-02  0.05  
3   1.4e+00  4.8e-02  4.5e-02  4.29e-01   -4.663611017e+00  -5.221056318e+00  4.8e-02  0.05  
4   3.5e-01  1.2e-02  4.1e-03  8.76e-01   -6.354495260e-01  -6.856359437e-01  1.2e-02  0.06  
5   6.4e-02  2.1e-03  4.0e-04  1.44e+00   -3


SDP solving time: 0.1503011 seconds.
optimum = -7.259477096422906e-8
Starting to compute the block structure...
No higher TSSOS hierarchy!
SEED=100139632
************************TSSOS************************
TSSOS is launching...
Starting to compute the block structure...
------------------------------------------------------
The sizes of PSD blocks:
[41, 8]
[1, 1]
------------------------------------------------------
Obtained the block structure. The maximal size of blocks is 41.
Assembling the SDP...
There are 525 affine constraints.
Solving the SDP...
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 525             
  Cones                  : 0               
  Scalar variables       : 1               
  Matrix variables       : 2               
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency che

Optimizer  - Semi-definite variables: 2                 scalarized             : 897             
Factor     - setup time             : 0.00              dense det. time        : 0.00            
Factor     - ML order time          : 0.00              GP order time          : 0.00            
Factor     - nonzeros before factor : 1.38e+05          after factor           : 1.38e+05        
Factor     - dense dim.             : 0                 flops                  : 5.04e+07        
ITE PFEAS    DFEAS    GFEAS    PRSTATUS   POBJ              DOBJ              MU       TIME  
0   2.6e+01  1.0e+00  1.0e+00  0.00e+00   0.000000000e+00   0.000000000e+00   1.0e+00  0.01  
1   1.1e+01  4.1e-01  6.2e-01  -9.68e-01  -1.578479174e+00  -2.854481020e+00  4.1e-01  0.03  
2   1.6e+00  5.9e-02  1.9e-01  -8.82e-01  -3.550529091e+01  -4.461198949e+01  5.9e-02  0.03  
3   1.1e+00  4.2e-02  5.0e-02  1.23e+00   -4.276737545e+00  -5.378825394e+00  4.2e-02  0.05  
4   1.6e-01  6.1e-03  1.7e-03  5.79e-01 

14  8.8e-08  1.9e-09  3.4e-13  1.04e+00   -2.804681440e-07  -3.066350383e-07  1.8e-09  0.14  
15  4.2e-08  2.1e-09  6.0e-14  9.98e-01   -8.895178276e-08  -9.828332936e-08  5.4e-10  0.14  
Optimizer terminated. Time: 0.17    

SDP solving time: 0.1770742 seconds.
optimum = -8.895178276215226e-8
Starting to compute the block structure...
No higher TSSOS hierarchy!
SEED=1001811023
************************TSSOS************************
TSSOS is launching...
Starting to compute the block structure...
------------------------------------------------------
The sizes of PSD blocks:
[41, 8]
[1, 1]
------------------------------------------------------
Obtained the block structure. The maximal size of blocks is 41.
Assembling the SDP...
There are 525 affine constraints.
Solving the SDP...
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 525             
  Cones  

...

Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.00    
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 525             
  Cones                  : 0               
  Scalar variables       : 1               
  Matrix variables       : 2               
  Integer variables      : 0               

Optimizer  - threads                : 4               
Optimizer  - solved problem         : the primal      
Optimizer  - Constraints            : 525
Optimizer  - Cones                  : 1
Optimizer  - Scalar variables       : 2                 conic                  : 2               
Optimizer  - Semi-def

5   2.0e-02  6.0e-04  4.2e-05  1.18e+00   -7.892534429e-02  -8.126054837e-02  6.0e-04  0.11  
6   3.2e-03  9.8e-05  3.0e-06  1.05e+00   -1.497020562e-02  -1.554351374e-02  9.8e-05  0.11  
7   4.3e-04  1.3e-05  1.8e-07  1.00e+00   -2.071880635e-03  -2.205818473e-03  1.3e-05  0.13  
8   7.1e-05  2.2e-06  1.2e-08  9.18e-01   -1.650494114e-04  -1.855721875e-04  2.2e-06  0.13  
9   2.3e-05  7.3e-07  2.4e-09  1.02e+00   -9.432257964e-05  -1.019585788e-04  7.3e-07  0.14  
10  4.3e-06  1.3e-07  1.9e-10  1.04e+00   -1.420113446e-05  -1.561153399e-05  1.3e-07  0.14  
11  7.3e-07  2.2e-08  1.3e-11  9.82e-01   -2.261095282e-06  -2.493622403e-06  2.2e-08  0.16  
12  1.4e-07  3.8e-09  8.7e-13  8.61e-01   -2.900190500e-07  -3.253043069e-07  3.8e-09  0.16  
13  2.7e-08  6.5e-10  6.2e-14  1.12e+00   -3.373437496e-08  -4.057740480e-08  7.0e-10  0.20  
Optimizer terminated. Time: 0.20    

SDP solving time: 0.2070096 seconds.
optimum = -3.3734374963711256e-8
Starting to compute the block structure...
No 

Excessive output truncated after 524292 bytes.

Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.00    
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 525             
  Cones                  : 0               
  Scalar variables       : 1               
  Matrix variables       : 2               
  Integer variables      : 0               

Optimizer  - threads                : 4               
Optimizer  - solved problem         : the primal      
Optimizer  - Constraints            : 525
Optimizer  - Cones                  : 1
Optimizer  - Scalar variables       : 2                 conic  

112/21.6k [02:46<08:54:27, 1s/it]
0.5%┣▏                                       ┫ 113/21.6k [02:47<08:52:58, 1s/it]
0.5%┣▏                                       ┫ 114/21.6k [02:48<08:51:23, 1s/it]
0.5%┣▏                                       ┫ 115/21.6k [02:49<08:49:43, 1s/it]
0.5%┣▏                                       ┫ 116/21.6k [02:50<08:48:27, 1s/it]
0.5%┣▏                                       ┫ 117/21.6k [02:51<08:47:03, 1s/it]
0.5%┣▏                                       ┫ 118/21.6k [02:52<08:45:52, 1s/it]
0.6%┣▏                                       ┫ 119/21.6k [02:53<08:44:15, 1s/it]
0.6%┣▎                                       ┫ 120/21.6k [02:54<08:42:53, 1s/it]
0.6%┣▎                                       ┫ 121/21.6k [02:55<08:41:38, 1s/it]
0.6%┣▎                                       ┫ 122/21.6k [02:56<08:40:29, 1s/it]
0.6%┣▎                                       ┫ 123/21.6k [02:57<08:39:15, 1s/it]


In [10]:
println("Noise levels")

h5open(output_file_name, "r") do file
        println(keys(file))
    end

Noise levels
["0.0", "0.01", "0.02", "0.04", "0.05"]


In [13]:
w = "0.05"
seeds = get_seeds(output_file_name, w)
println(length(seeds))

20032


In [12]:
seed = seeds[3]

println("For w="*w*" and seed="*seed)

h5open(output_file_name, "r") do file
       println("First hierarcy opminum:")
       println(read(file[w][seed]["first_opt"]))
       println("Best opminum:")
       println(read(file[w][seed]["best_opt"]))
       println("Last hierarcy numer:")
       println(read(file[w][seed]["iter"]))
end

For w=0.05 and seed=1001165252
First hierarcy opminum:
0.05188472943719246
Best opminum:
0.05188472943719246
Last hierarcy numer:
1
